# AIRD 데이터 품질 진단 시스템

서울 공장등록 데이터의 품질을 5대 차원으로 진단하고 HTML 리포트를 생성합니다.

In [ ]:
import os

def is_colab():
    """현재 실행 환경이 Google Colab인지 확인"""
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    print("▶ Colab 환경입니다.")

    # ------------------------
    # 1) GitHub Repo Clone
    # ------------------------
    repo_url = "https://github.com/hike-lab/AIRD-PACK.git"
    repo_name = repo_url.split('/')[-1].replace('.git', '')

    if os.path.exists(repo_name):
        print(f"이미 '{repo_name}' 폴더가 있어 clone을 스킵합니다.")
    else:
        print("GitHub repo clone 중...")
        !git clone $repo_url
        print("clone 완료!")

    # ------------------------
    # 2) 한글 폰트 설치
    # ------------------------
    print("나눔 폰트 설치 중...")
    !sudo apt-get install -y fonts-nanum
    !sudo fc-cache -fv
    !rm ~/.cache/matplotlib -rf
    print("폰트 설치 완료! (런타임 재시작 필요)")

    # ------------------------
    # 3) requirements 설치
    # ------------------------
    print("requirements.txt 설치 중...")
    %cd $repo_name
    !pip install -r requirements.txt
    %cd ..

    # ------------------------
    # 4) Colab용 경로 설정
    # ------------------------
    RAW_DATA_PATH = f"{repo_name}/data/raw/"
    PROCESSED_DATA_PATH = f"{repo_name}/data/processed/"
    OUTPUT_PATH = f"{repo_name}/code/ml-pack/outputs/"

    print("폴더 경로 설정 완료!")
    print("RAW_DATA_PATH:", RAW_DATA_PATH)
    print("PROCESSED_DATA_PATH:", PROCESSED_DATA_PATH)
    print("OUTPUT_PATH:", OUTPUT_PATH)

else:
    RAW_DATA_PATH = "../../data/raw/"
    PROCESSED_DATA_PATH = "../../data/processed/"
    OUTPUT_PATH = "../../code/ml-pack/outputs/"
    print("▶ 로컬 환경입니다. git clone 및 폰트 설치는 실행하지 않습니다.")

    print("로컬 경로 설정 완료!")
    print("RAW_DATA_PATH:", RAW_DATA_PATH)
    print("PROCESSED_DATA_PATH:", PROCESSED_DATA_PATH)
    print("OUTPUT_PATH:", OUTPUT_PATH)

▶ Colab 환경입니다.
이미 'AIRD-PACK' 폴더가 있어 clone을 스킵합니다.
나눔 폰트 설치 중...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberatio

In [ ]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 폰트 설정
import matplotlib.font_manager as fm


def set_korean_font():
    """
    플랫폼 판별 및 폰트 다운로드 없이
    matplotlib에 이미 설치된 한글 폰트만 사용
    """

    candidate_fonts = [
        "Malgun Gothic",      # Windows
        "AppleGothic",        # macOS
        "NanumGothic",        # Linux / 일부 Colab
        "Noto Sans CJK KR",
        "Noto Sans KR"
    ]

    installed_fonts = {f.name for f in fm.fontManager.ttflist}
    for font in candidate_fonts:
        if font in installed_fonts:
            plt.rcParams["font.family"] = font
            plt.rcParams["axes.unicode_minus"] = False
            print(f"✅ 한글 폰트 적용: {font}")
            return

# 실행
set_korean_font()

print("✅ 라이브러리 로드 완료")

✅ 한글 폰트 적용: NanumGothic
✅ 라이브러리 로드 완료


In [ ]:
# 데이터 임포트
try:
    df = pd.read_csv(f'{RAW_DATA_PATH}seoul_factory_registry_2025_v1.csv')
    print(f"✅ 데이터 로드 성공")
    print(f"데이터 크기: {df.shape[0]:,}행 × {df.shape[1]}컬럼")
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {RAW_DATA_PATH}seoul_factory_registry_2025_v1.csv")
    print("현재 위치를 확인하세요!")
    import os
    print(f"현재 위치: {os.getcwd()}")
    raise

✅ 데이터 로드 성공
데이터 크기: 33,096행 × 80컬럼


In [ ]:
# 품질 진단 결과 저장 딕셔너리
quality_results = {}

# 1. 완전성 (Completeness)
print("\n" + "="*60)
print("1️⃣ 완전성 진단 (Completeness)")
print("="*60)

missing_stats = []
for col in df.columns:
    total = len(df)
    missing = df[col].isna().sum()
    missing_pct = (missing / total) * 100
    missing_stats.append({
        '컬럼명': col,
        '결측수': missing,
        '결측비율(%)': round(missing_pct, 2),
        '완전성점수': 100 - missing_pct
    })

missing_df = pd.DataFrame(missing_stats).sort_values('결측비율(%)', ascending=False)
completeness_score = missing_df['완전성점수'].mean()

print(f"\n완전성 종합 점수: {completeness_score:.1f}/100")
print(f"\n결측치 상위 5개 컬럼:")

quality_results['completeness'] = {
    'score': completeness_score,
    'details': missing_df
}
missing_df.head()


1️⃣ 완전성 진단 (Completeness)

완전성 종합 점수: 98.2/100

결측치 상위 5개 컬럼:


,컬럼명,결측수,결측비율(%),완전성점수
54,코드매핑_법인주소,31417,94.93,5.073121
78,정제_최초승인일,16759,50.64,49.362461
74,코드매핑_지목,183,0.55,99.447063
73,정제_지목,183,0.55,99.447063
51,정제_등록일,29,0.09,99.912376


In [ ]:
# 2. 정확성 (Accuracy)
print("\n" + "="*60)
print("2️⃣ 정확성 진단 (Accuracy)")
print("="*60)

accuracy_issues = []

# 날짜 형식 검증
date_cols = ['최초승인일', '최초등록일', '등록일']
for col in date_cols:
    if col in df.columns:
        valid = pd.to_datetime(df[col], format='%Y%m%d', errors='coerce').notna().sum()
        total = df[col].notna().sum()
        if total > 0:
            accuracy = (valid / total) * 100
            accuracy_issues.append({
                '항목': f'{col} 형식',
                '정확도(%)': round(accuracy, 2)
            })

# 숫자 컬럼 검증
numeric_cols = ['용지면적', '제조시설면적', '건축면적']
for col in numeric_cols:
    if col in df.columns:
        valid = pd.to_numeric(df[col], errors='coerce').notna().sum()
        total = df[col].notna().sum()
        if total > 0:
            accuracy = (valid / total) * 100
            accuracy_issues.append({
                '항목': f'{col} 숫자형식',
                '정확도(%)': round(accuracy, 2)
            })

accuracy_df = pd.DataFrame(accuracy_issues)
accuracy_score = accuracy_df['정확도(%)'].mean() if len(accuracy_df) > 0 else 100

print(f"\n정확성 종합 점수: {accuracy_score:.1f}/100")
if len(accuracy_df) > 0:
    print(f"\n형식 검증 결과:")
    display(accuracy_df)

quality_results['accuracy'] = {
    'score': accuracy_score,
    'details': accuracy_df
}


2️⃣ 정확성 진단 (Accuracy)

정확성 종합 점수: 91.5/100

형식 검증 결과:


,항목,정확도(%)
0,최초승인일 형식,49.36
1,최초등록일 형식,99.93
2,등록일 형식,99.91
3,용지면적 숫자형식,100.00
4,제조시설면적 숫자형식,100.00
5,건축면적 숫자형식,100.00


In [ ]:
# 3. 일관성 (Consistency)
print("\n" + "="*60)
print("3️⃣ 일관성 진단 (Consistency)")
print("="*60)

consistency_checks = []

# 중복 검사
if '공장관리번호' in df.columns:
    duplicates = df['공장관리번호'].duplicated().sum()
    dup_rate = (duplicates / len(df)) * 100
    consistency_checks.append({
        '검사항목': '중복 레코드',
        '이상건수': duplicates,
        '이상비율(%)': round(dup_rate, 2),
        '일관성점수': 100 - dup_rate
    })

# 시도명 일관성
if '시도명' in df.columns:
    non_seoul = (df['시도명'] != '서울특별시').sum()
    non_seoul_rate = (non_seoul / len(df)) * 100
    consistency_checks.append({
        '검사항목': '시도명 일관성',
        '이상건수': non_seoul,
        '이상비율(%)': round(non_seoul_rate, 2),
        '일관성점수': 100 - non_seoul_rate
    })

consistency_df = pd.DataFrame(consistency_checks)
consistency_score = consistency_df['일관성점수'].mean() if len(consistency_df) > 0 else 100

print(f"\n일관성 종합 점수: {consistency_score:.1f}/100")
if len(consistency_df) > 0:
    print(f"\n일관성 검사 결과:")
    display(consistency_df)

quality_results['consistency'] = {
    'score': consistency_score,
    'details': consistency_df
}


3️⃣ 일관성 진단 (Consistency)

일관성 종합 점수: 67.0/100

일관성 검사 결과:


,검사항목,이상건수,이상비율(%),일관성점수
0,중복 레코드,21870,66.08,33.919507
1,시도명 일관성,0,0.00,100.000000


In [ ]:
# 4. 적시성 (Timeliness)
print("\n" + "="*60)
print("4️⃣ 적시성 진단 (Timeliness)")
print("="*60)

# 데이터 최신성 평가
current_year = 2025
timeliness_checks = []

if '최초승인일' in df.columns:
    df_temp = df['최초승인일'].dropna()
    if len(df_temp) > 0:
        dates = pd.to_datetime(df_temp, format='%Y%m%d', errors='coerce')
        valid_dates = dates.dropna()
        if len(valid_dates) > 0:
            years_old = current_year - valid_dates.dt.year
            avg_age = years_old.mean()
            # 10년 이내면 100점, 그 이상은 감점
            timeliness_score = max(0, 100 - (avg_age - 10) * 5)
            timeliness_checks.append({
                '항목': '데이터 평균 연령',
                '값': f'{avg_age:.1f}년',
                '적시성점수': round(timeliness_score, 1)
            })

timeliness_df = pd.DataFrame(timeliness_checks)
timeliness_score = timeliness_df['적시성점수'].mean() if len(timeliness_df) > 0 else 80

print(f"\n적시성 종합 점수: {timeliness_score:.1f}/100")
if len(timeliness_df) > 0:
    print(f"\n적시성 평가:")
    display(timeliness_df)

quality_results['timeliness'] = {
    'score': timeliness_score,
    'details': timeliness_df
}


4️⃣ 적시성 진단 (Timeliness)

적시성 종합 점수: 79.9/100

적시성 평가:


,항목,값,적시성점수
0,데이터 평균 연령,14.0년,79.9


In [ ]:
# 5. 유효성 (Validity)
print("\n" + "="*60)
print("5️⃣ 유효성 진단 (Validity)")
print("="*60)

validity_checks = []

# 면적 값 유효성
area_cols = ['용지면적', '제조시설면적', '건축면적']
for col in area_cols:
    if col in df.columns:
        numeric_vals = pd.to_numeric(df[col], errors='coerce')
        valid_vals = numeric_vals[(numeric_vals > 0) & (numeric_vals < 1000000)]
        total = numeric_vals.notna().sum()
        if total > 0:
            validity_rate = (len(valid_vals) / total) * 100
            validity_checks.append({
                '항목': f'{col} 범위',
                '유효비율(%)': round(validity_rate, 2)
            })

validity_df = pd.DataFrame(validity_checks)
validity_score = validity_df['유효비율(%)'].mean() if len(validity_df) > 0 else 100

print(f"\n유효성 종합 점수: {validity_score:.1f}/100")
if len(validity_df) > 0:
    print(f"\n유효성 검사 결과:")
    display(validity_df)

quality_results['validity'] = {
    'score': validity_score,
    'details': validity_df
}


5️⃣ 유효성 진단 (Validity)

유효성 종합 점수: 91.3/100

유효성 검사 결과:


,항목,유효비율(%)
0,용지면적 범위,74.77
1,제조시설면적 범위,99.43
2,건축면적 범위,99.66


In [ ]:
# 종합 품질 점수 계산
print("\n" + "="*60)
print("📊 종합 품질 점수")
print("="*60)

dimension_scores = {
    '완전성': completeness_score,
    '정확성': accuracy_score,
    '일관성': consistency_score,
    '적시성': timeliness_score,
    '유효성': validity_score
}

total_score = sum(dimension_scores.values()) / len(dimension_scores)

print(f"\n【 5대 차원별 점수 】")
for dim, score in dimension_scores.items():
    print(f"  {dim}: {score:.1f}/100")

print(f"\n【 종합 품질 점수 】")
print(f"  ⭐ {total_score:.1f}/100")

if total_score >= 90:
    grade = "A (우수)"
elif total_score >= 80:
    grade = "B (양호)"
elif total_score >= 70:
    grade = "C (보통)"
else:
    grade = "D (개선필요)"

print(f"  등급: {grade}")

quality_results['total'] = {
    'score': total_score,
    'grade': grade,
    'dimensions': dimension_scores
}


📊 종합 품질 점수

【 5대 차원별 점수 】
  완전성: 98.2/100
  정확성: 91.5/100
  일관성: 67.0/100
  적시성: 79.9/100
  유효성: 91.3/100

【 종합 품질 점수 】
  ⭐ 85.6/100
  등급: B (양호)


In [ ]:
# HTML 리포트 생성
print("\n" + "="*60)
print("📄 HTML 리포트 생성 중...")
print("="*60)

html_content = f"""
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AIRD 데이터 품질 진단 리포트</title>
    <style>
        * {{ margin: 0; padding: 0; box-sizing: border-box; }}
        body {{
            font-family: 'Segoe UI', Arial, sans-serif;
            line-height: 1.6;
            color: #333;
            background: #f5f7fa;
            padding: 20px;
        }}
        .container {{
            max-width: 1200px;
            margin: 0 auto;
            background: white;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            overflow: hidden;
        }}
        .header {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 40px;
            text-align: center;
        }}
        .header h1 {{ font-size: 2.5em; margin-bottom: 10px; }}
        .header p {{ font-size: 1.1em; opacity: 0.9; }}
        .content {{ padding: 40px; }}
        .summary-box {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 10px;
            text-align: center;
            margin-bottom: 30px;
        }}
        .summary-box .score {{
            font-size: 4em;
            font-weight: bold;
            margin: 10px 0;
        }}
        .summary-box .grade {{
            font-size: 1.5em;
            opacity: 0.9;
        }}
        .dimensions {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }}
        .dimension-card {{
            background: #f8f9fa;
            padding: 20px;
            border-radius: 8px;
            border-left: 4px solid #667eea;
        }}
        .dimension-card h3 {{ color: #667eea; margin-bottom: 10px; }}
        .dimension-card .score {{ font-size: 2em; font-weight: bold; color: #333; }}
        .section {{ margin-bottom: 40px; }}
        .section h2 {{
            color: #667eea;
            border-bottom: 2px solid #667eea;
            padding-bottom: 10px;
            margin-bottom: 20px;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin-top: 15px;
        }}
        th, td {{
            padding: 12px;
            text-align: left;
            border-bottom: 1px solid #dee2e6;
        }}
        th {{
            background-color: #667eea;
            color: white;
            font-weight: 600;
        }}
        tr:hover {{ background-color: #f8f9fa; }}
        .footer {{
            text-align: center;
            padding: 20px;
            background: #f8f9fa;
            color: #6c757d;
        }}
        .badge {{
            display: inline-block;
            padding: 5px 10px;
            border-radius: 4px;
            font-size: 0.9em;
            font-weight: 600;
        }}
        .badge-success {{ background: #28a745; color: white; }}
        .badge-warning {{ background: #ffc107; color: #333; }}
        .badge-danger {{ background: #dc3545; color: white; }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🔍 AIRD 데이터 품질 진단 리포트</h1>
            <p>서울특별시 공장등록 데이터 품질 분석</p>
            <p>생성일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
        </div>

        <div class="content">
            <div class="summary-box">
                <h2>종합 품질 점수</h2>
                <div class="score">{total_score:.1f}</div>
                <div class="grade">{grade}</div>
            </div>

            <div class="section">
                <h2>📊 데이터 개요</h2>
                <table>
                    <tr><th>항목</th><th>값</th></tr>
                    <tr><td>총 레코드 수</td><td>{len(df):,}행</td></tr>
                    <tr><td>총 컬럼 수</td><td>{len(df.columns)}개</td></tr>
                    <tr><td>파일 크기</td><td>약 {df.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB</td></tr>
                </table>
            </div>

            <div class="section">
                <h2>⭐ 5대 품질 차원</h2>
                <div class="dimensions">
                    <div class="dimension-card">
                        <h3>1️⃣ 완전성</h3>
                        <div class="score">{completeness_score:.1f}</div>
                        <p>결측치 및 데이터 누락 평가</p>
                    </div>
                    <div class="dimension-card">
                        <h3>2️⃣ 정확성</h3>
                        <div class="score">{accuracy_score:.1f}</div>
                        <p>데이터 형식 및 유효성 평가</p>
                    </div>
                    <div class="dimension-card">
                        <h3>3️⃣ 일관성</h3>
                        <div class="score">{consistency_score:.1f}</div>
                        <p>중복 및 일관성 평가</p>
                    </div>
                    <div class="dimension-card">
                        <h3>4️⃣ 적시성</h3>
                        <div class="score">{timeliness_score:.1f}</div>
                        <p>데이터 최신성 평가</p>
                    </div>
                    <div class="dimension-card">
                        <h3>5️⃣ 유효성</h3>
                        <div class="score">{validity_score:.1f}</div>
                        <p>값의 범위 및 타당성 평가</p>
                    </div>
                </div>
            </div>

            <div class="section">
                <h2>🔍 완전성 상세 (결측치 상위 10개)</h2>
                <table>
                    <tr>
                        <th>컬럼명</th>
                        <th>결측 수</th>
                        <th>결측 비율</th>
                        <th>상태</th>
                    </tr>
"""

# 결측치 상위 10개 추가
for _, row in missing_df.head(10).iterrows():
    if row['결측비율(%)'] > 50:
        badge_class = "badge-danger"
    elif row['결측비율(%)'] > 20:
        badge_class = "badge-warning"
    else:
        badge_class = "badge-success"

    html_content += f"""
                    <tr>
                        <td>{row['컬럼명']}</td>
                        <td>{row['결측수']:,}</td>
                        <td>{row['결측비율(%)']:.2f}%</td>
                        <td><span class="badge {badge_class}">{row['결측비율(%)']:.1f}%</span></td>
                    </tr>
"""

html_content += """
                </table>
            </div>
        </div>

        <div class="footer">
            <p>AIRD Pack v1.0 | AI Ready Data Quality Diagnosis System</p>
            <p>© 2025 AIRD Project</p>
        </div>
    </div>
</body>
</html>
"""

# HTML 파일 저장
import os
os.makedirs(f'{OUTPUT_PATH}reports', exist_ok=True)

output_path = f'{OUTPUT_PATH}reports/quality_report.html'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"\n✅ HTML 리포트 생성 완료!")
print(f"저장 위치: {output_path}")
print(f"\n브라우저에서 파일을 열어보세요!")


📄 HTML 리포트 생성 중...

✅ HTML 리포트 생성 완료!
저장 위치: AIRD-PACK/code/ml-pack/outputs/reports/quality_report.html

브라우저에서 파일을 열어보세요!


In [ ]:
# 완료 메시지
print("\n" + "="*60)
print("🎉 품질 진단 완료!")
print("="*60)
print(f"\n종합 점수: {total_score:.1f}/100 ({grade})")
print(f"\n다음 단계: aird_ml_factory_pack_builder.ipynb 실행")


🎉 품질 진단 완료!

종합 점수: 85.6/100 (B (양호))

다음 단계: aird_ml_factory_pack_builder.ipynb 실행
